In [ ]:
# taken from https://towardsdatascience.com/yolo-object-detection-with-opencv-and-python-21e50ac599e9
# repo : https://github.com/arunponnusamy/object-detection-opencv

import cv2
import numpy as np
import imutils
from IPython.display import Image, display

# functions
def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers


def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

scale = 0.00392

# read class names from text file
classes = None
with open('yolov3.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# read pre-trained model and config file
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')



# cap = cv2.VideoCapture('/outputs/jitter-10/clips/105-0-0-0.mp4')
# cap = cv2.VideoCapture('/outputs/jitter-10/clips/3-0-0-0.mp4')
# cap = cv2.VideoCapture('/outputs/jitter-10/clips/923-0-0-0.mp4')
cap = cv2.VideoCapture('/outputs/jitter-10/clips/1551-0-0-0.mp4')

counter = 0
while cap.isOpened():
    ret, image = cap.read()

    if counter == 600:
        break
    counter += 1

    Width = image.shape[1]
    Height = image.shape[0]   

    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

    # set input blob for the network
    net.setInput(blob)

    outs = net.forward(get_output_layers(net))

    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4


    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.2:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])


    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    
    for i in indices:
        i = i[0]
        if class_ids[i] == 0:
            print('pedestrian is present')


